# Results Notebook

In [1]:
%load_ext autoreload
%autoreload 2

import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import seaborn as sns
import pandas as pd
df = pd.DataFrame
sns.set_theme()

def melt_row_col_data(arr,name):
    arr_df = df(arr)
    arr_df = arr_df.melt(
        ignore_index=False,
        var_name='Column',
        value_name='ADC_OUT_AMS',
    )
    arr_df['Row'] = arr_df.index
    return arr_df

In [ ]:
# Generate the reference result

w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)
wx = w @ x.T
wx_rounded = np.round(wx.T / 2**2).astype(int).T[::-1].T

### Comparing AMS vs RTL simulations of QRAcc

In [ ]:
ams = np.loadtxt('../tb/qracc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/qracc/inputs/adc_out_rtlsims.txt')

ams_df = df(ams)
rtl_df = df(rtl)

# ams_df['Row'] = ams_df.index
# rtl_df['Row'] = rtl_df.index

ams_df = ams_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_AMS',
)
rtl_df = rtl_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_RTL',
)
ams_df['Row'] = ams_df.index
rtl_df['Row'] = rtl_df.index
# Join the two dataframes
adc_df = pd.merge(ams_df, rtl_df)

rmse = np.sqrt(np.mean((rtl.flatten() - ams.flatten())**2))
fig = px.scatter(adc_df, y='ADC_OUT_AMS', x='ADC_OUT_RTL', width=400, height=400, title=f'AMS vs Shifted, RMSE={rmse:.2f}', labels={'ADC_OUT_AMS':'AMS','ADC_OUT_RTL':'RTL'})
fig.show()
fig.write_image('../images/rtl_vs_ams.svg')

# Sequential Accelerator Simulations

In [ ]:
w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 3, #nColumns
    wDimY = 4, #nRows
    xBatches = 10,
    xTrits = 4,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)

# ams = np.loadtxt('../tb/seq_acc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/seq_acc/inputs/mac_out_rtlsims.txt')

wx = w @ x.T

rtl.astype(int), wx[::-1].T
rmse = np.sqrt(np.mean((rtl.flatten() - wx[::-1].T.flatten())**2))
px.scatter(x=rtl.flatten(), y=wx[::-1].T.flatten(), width=400, height=400, title=f'AMS vs Shifted, RMSE={rmse:.2f}')